In [22]:
# Goal:
## 1: pull 100 random public matches.
## 2: get 100 match details.
## 3: get 10 all player MMRs from al those matches (max 1000).
## 4: put those data into PostGres
# Relies on PostGres DB being up and running on localhost
import requests
import pandas as pd
import numpy as np
import psycopg2
import json
import time

In [23]:
# get random set of 100 matches
response = requests.get("https://api.opendota.com/api/publicMatches")
public_match_results = pd.DataFrame.from_dict(response.json(), orient='columns')

In [24]:
# clean up Match data
## NaN MMR is unhelpful
## fewer than 10 players is unhelpful
public_match_results.dropna(inplace=True)

# Low-tech check for 5v5? Check for 4 commas (separates 5 hero values) on each team
public_match_results['commacount'] = public_match_results.apply(lambda x: x.radiant_team.count(',')+x.dire_team.count(','),axis=1)
public_match_resultsn = public_match_results[public_match_results['commacount'] == 8]
public_match_results.drop(['commacount'],axis=1, inplace=True)

# hero names not needed, these will be pulled in separately with Players later
public_match_results.drop(['radiant_team','dire_team'],axis=1,inplace=True)

In [25]:
batch_file_name = 'last_matchresults_batch.csv'
public_match_results.to_csv(batch_file_name,header=None,index=None,sep='|')

In [26]:
# match data is prepared. time to move to Postgres
## get credentials from .config, connect to DB
credential_dict = {}
with open ('connections.config','r') as config_file:
    credential_dict = json.loads(config_file.read())

connection_string = "dbname="+credential_dict['psql_dbname'] + \
    " user="+credential_dict['psql_username'] + \
    " password="+credential_dict['psql_password']

conn = psycopg2.connect(connection_string)

# place the psql cursor
cur = conn.cursor()

In [27]:
# copy csv to Postgres, commit the change
## Need to address duplicate primary keys earlier in process - or in copy_from
with open (batch_file_name,'r') as batch_file:
    cur.copy_from(batch_file, 'match_results', sep='|')
conn.commit()

UniqueViolation: duplicate key value violates unique constraint "match_results_pkey"
DETAIL:  Key (match_id)=(7216020117) already exists.
CONTEXT:  COPY match_results, line 15


In [28]:
# close psql cursor and connection
cur.close()
conn.close()

In [ ]:
# 2023-06-25 starting with just one match from this list.
## Ultimately, there will be loops to accomplish each step.
#target_matches = []
#target_matches.append(public_match_results.iloc[0]['match_id']) # just grab the first match for now
#match_details = pd.DataFrame()

# the following code doesn't work. The length of Match dicts are inconsistent.
# will need to evaluate these inconsistencies deeper and then return here.
#for i,target_match in enumerate(target_matches):
#    # target matches are numpy.int64 by default
#    api_string = "https://api.opendota.com/api/matches/"+str(target_match)
#    response = requests.get(api_string)
##    if response.status_code == 200:
#        if i == 0:
#            match_details = pd.DataFrame.from_dict(response.json(), orient='columns') # arrays not of same length
#        else:
#            pass # to be replaced later with Concatenate
#    else:
#        print("Response status code: ", response.status_code)
#        break

In [ ]:
#response.json()

In [ ]:
#response.json()['players'][0]

In [ ]:
#response2.json()['players'][0]['account_id']

In [ ]:
#need player details as well for MMR etc.
#response3 = requests.get("https://api.opendota.com/api/players/416336446")
#response3.json()

In [ ]:
# There's the MMR and also the Rank Tier above. Will take some work to get 100 matches, then 100 match results, then 10 players per match result to get all MMRs and rank tiers.


In [ ]:
# for spacing out API calls
#print('test')
#time.sleep(2)
#print('test2')